In [4]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Define the stocks and weights
stocks = ['ASTS', 'LUMN', 'SMMT', 'VKTX', '196170.KQ', '0020.HK']

# Download the adjusted close price data for the last year
end_date = "2024-10-09"
start_date = "2023-10-09"
data = yf.download(stocks, start=start_date, end=end_date)['Adj Close']

# Calculate daily returns
returns = data.pct_change().dropna()

# Define a function to calculate portfolio performance
def portfolio_performance(weights, returns):
    portfolio_return = np.sum(returns.mean() * weights) * 252
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    sharpe_ratio = portfolio_return / portfolio_volatility
    return portfolio_return, portfolio_volatility, sharpe_ratio

# Define the objective function (negative Sharpe ratio for minimization)
def negative_sharpe_ratio(weights, returns):
    return -portfolio_performance(weights, returns)[2]

# Constraints: sum of weights is 1
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Bounds: each weight is between 0 and 1
bounds = [(0, 1)] * len(stocks)

# Initial guess (equal distribution)
initial_weights = [1 / len(stocks)] * len(stocks)

# Optimize the portfolio for maximum Sharpe ratio
optimized_result = minimize(negative_sharpe_ratio, initial_weights, args=(returns,), method='SLSQP', bounds=bounds, constraints=constraints)

# Get the optimized weights and performance metrics
optimized_weights = optimized_result.x
optimized_return, optimized_volatility, optimized_sharpe_ratio = portfolio_performance(optimized_weights, returns)

# Display results
optimized_weights, optimized_return, optimized_volatility, optimized_sharpe_ratio


[*********************100%***********************]  6 of 6 completed
C:\Users\Alexander\AppData\Local\Temp\ipykernel_6740\1302828425.py:15: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change().dropna()


(array([0.07848169, 0.43567611, 0.13350179, 0.12647187, 0.07699314,
        0.14887541]),
 2.30405257420251,
 0.5834691276556188,
 3.948885150890849)